d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

-sandbox
# Model Registry

MLflow Model Registry is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance.  This lesson explores how to manage models using the MLflow model registry.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Register a model using MLflow
 - Deploy that model into production
 - Update a model in production to new version including a staging phase for testing
 - Archive and delete models
 
## Prerequisites
- Web browser: Chrome
- A cluster configured with **8 cores** and **DBR 7.0 ML**

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> This notebook makes use of the model registry. If you are using the Databricks Community Edition, click the `Upgrade` button on the landing page <a href="https://accounts.cloud.databricks.com/registration.html#login" target="_blank">or navigate here</a> to start a free trial.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the<br/>
start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [0]:
%run "./Includes/Classroom-Setup"

-sandbox
### Model Registry

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. It provides model lineage (which MLflow Experiment and Run produced the model), model versioning, stage transitions (e.g. from staging to production), annotations (e.g. with comments, tags), and deployment management (e.g. which production jobs have requested a specific model version).

Model registry has the following features:<br><br>

* **Central Repository:** Register MLflow models with the MLflow Model Registry. A registered model has a unique name, version, stage, and other metadata.
* **Model Versioning:** Automatically keep track of versions for registered models when updated.
* **Model Stage:** Assigned preset or custom stages to each model version, like “Staging” and “Production” to represent the lifecycle of a model.
* **Model Stage Transitions:** Record new registration events or changes as activities that automatically log users, changes, and additional metadata such as comments.
* **CI/CD Workflow Integration:** Record stage transitions, request, review and approve changes as part of CI/CD pipelines for better control and governance.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/model-registry.png" style="height: 400px; margin: 20px"/></div>

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See <a href="https://mlflow.org/docs/latest/registry.html" target="_blank">the MLflow docs</a> for more details on the model registry.

-sandbox
### Registering a Model

The following workflow will work with either the UI or in pure Python.  This notebook will use pure Python.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Explore the UI throughout this lesson by clicking the "Models" tab on the left-hand side of the screen.

Confirm that you are using `mlflow` version 1.7 or higher.

In [0]:
from distutils.version import LooseVersion, StrictVersion
import mlflow 

assert StrictVersion(mlflow.__version__) >= StrictVersion("1.7.0"), "Update MLflow to version 1.7.0+"

Train a model and log it to MLflow.

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv")
X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

rf = RandomForestRegressor(n_estimators=100, max_depth=5)
rf.fit(X_train, y_train)

with mlflow.start_run(run_name="RF Model") as run:
  mlflow.sklearn.log_model(rf, "model")
  mlflow.log_metric("mse", mean_squared_error(y_test, rf.predict(X_test)))

  runID = run.info.run_uuid

Create a unique model name so you don't clash with other workspace users.

In [0]:
import uuid

model_name = f"airbnb_rf_model_{uuid.uuid4().hex[:10]}"
model_name

Register the model.

In [0]:
model_uri = "runs:/{run_id}/model".format(run_id=runID)

model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

-sandbox
 **Open the *Models* tab on the left of the screen to explore the registered model.**  Note the following:<br><br>

* It logged who trained the model and what code was used
* It logged a history of actions taken on this model
* It logged this model as a first version

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/model-registry-1.png" style="height: 400px; margin: 20px"/></div>

Check the status.  It will initially be in `PENDING_REGISTRATION` status.

In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()
model_version_details = client.get_model_version(name=model_name, version=1)

model_version_details.status

Now add a model description

In [0]:
client.update_registered_model(
  name=model_details.name,
  description="This model forecasts Airbnb housing list prices based on various listing inputs."
)

Add a version-specific description.

In [0]:
client.update_model_version(
  name=model_details.name,
  version=model_details.version,
  description="This model version was built using sklearn."
)

### Deploying a Model

The MLflow Model Registry defines several model stages: `None`, `Staging`, `Production`, and `Archived`. Each stage has a unique meaning. For example, `Staging` is meant for model testing, while `Production` is for models that have completed the testing or review processes and have been deployed to applications. 

Users with appropriate permissions can transition models between stages. In private preview, any user can transition a model to any stage. In the near future, administrators in your organization will be able to control these permissions on a per-user and per-model basis.

If you have permission to transition a model to a particular stage, you can make the transition directly by using the `MlflowClient.update_model_version()` function. If you do not have permission, you can request a stage transition using the REST API; for example: ```%sh curl -i -X POST -H "X-Databricks-Org-Id: <YOUR_ORG_ID>" -H "Authorization: Bearer <YOUR_ACCESS_TOKEN>" https://<YOUR_DATABRICKS_WORKSPACE_URL>/api/2.0/preview/mlflow/transition-requests/create -d '{"comment": "Please move this model into production!", "model_version": {"version": 1, "registered_model": {"name": "power-forecasting-model"}}, "stage": "Production"}'
```

Now that you've learned about stage transitions, transition the model to the `Production` stage.

In [0]:
import time

time.sleep(10) # In case the registration is still pending

In [0]:
client.update_model_version(
  name=model_details.name,
  version=model_details.version,
  stage='Production',
)

Fetch the model's current status.

In [0]:
model_version_details = client.get_model_version(
  name=model_details.name,
  version=model_details.version,
)
print("The current model stage is: '{stage}'".format(stage=model_version_details.current_stage))

-sandbox
Fetch the latest model using a `pyfunc`.  Loading the model in this way allows us to use the model regardless of the package that was used to train it.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You can load a specific version of the model too.

In [0]:
import mlflow.pyfunc

model_version_uri = "models:/{model_name}/1".format(model_name=model_name)

print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_version_uri))
model_version_1 = mlflow.pyfunc.load_model(model_version_uri)

Apply the model.

In [0]:
model_version_1.predict(X_test)

### Deploying a New Model Version

The MLflow Model Registry enables you to create multiple model versions corresponding to a single registered model. By performing stage transitions, you can seamlessly integrate new model versions into your staging or production environments.

Create a new model version and register that model when it's logged.

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

df = pd.read_csv("/dbfs/mnt/training/airbnb/sf-listings/airbnb-cleaned-mlflow.csv")
X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

rf = RandomForestRegressor(n_estimators=300, max_depth=10)
rf.fit(X_train, y_train)

with mlflow.start_run(run_name="RF Model") as run:
  # Specify the `registered_model_name` parameter of the `mlflow.sklearn.log_model()`
  # function to register the model with the MLflow Model Registry. This automatically
  # creates a new model version
  mlflow.sklearn.log_model(
    sk_model=rf,
    artifact_path="sklearn-model",
    registered_model_name=model_name,
  )
  mlflow.log_metric("mse", mean_squared_error(y_test, rf.predict(X_test)))

  runID = run.info.run_uuid

-sandbox
Check the UI to see the new model version.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-4/model-registry-2.png" style="height: 400px; margin: 20px"/></div>

Use the search functionality to grab the latest model version.

In [0]:
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])

Add a description to this new version.

In [0]:
client.update_model_version(
  name=model_name,
  version=new_model_version,
  description="This model version is a random forest containing 300 decision trees and a max depth of 10 that was trained in scikit-learn."
)

Put this new model version into `Staging`

In [0]:
import time

time.sleep(10) # In case the registration is still pending

In [0]:
client.update_model_version(
  name=model_name,
  version=new_model_version,
  stage="Staging",
)

Sicne this model is now in staging, you can execute an automated CI/CD pipeline against it to test it before going into production.  Once that is completed, you can push that model into production.

In [0]:
client.update_model_version(
  name=model_name,
  version=new_model_version,
  stage="Production",
)

### Archiving and Deleting

You can now archive and delete old versions of the model.

In [0]:
client.update_model_version(
  name=model_name,
  version=1,
  stage="Archived",
)

-sandbox
Delete version 1.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You cannot delete a model that is not first archived.

In [0]:
client.delete_model_version(
  name=model_name,
  version=1
)

Archive version 2 of the model too.

In [0]:
client.update_model_version(
  name=model_name,
  version=2,
  stage="Archived",
)

Now delete the entire registered model.

In [0]:
client.delete_registered_model(model_name)

## Review
**Question:** How does MLflow tracking differ from the model registry?  
**Answer:** Tracking is meant for experimentation and development.  The model registry is designed to take a model from tracking and put it through staging and into production.  This is often the point that a data engineer or a machine learning engineer takes responsibility for the depoloyment process.

**Question:** Why do I need a model registry?  
**Answer:** Just as MLflow tracking provides end-to-end reproducibility for the machine learning training process, a model registry provides reproducibility and governance for the deployment process.  Since production systems are mission critical, components can be isolated with ACL's so only specific individuals can alter production models.  Version control and CI/CD workflow integration is also a critical dimension of deploying models into production.

**Question:** What can I do programatically versus using the UI?  
**Answer:** Most operations can be done using the UI or in pure Python.  A model must be tracked using Python, but from that point on everything can be done either way.  For instance, a model logged using the MLflow tracking API can then be registered using the UI and can then be pushed into production.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [0]:
%run "./Includes/Classroom-Cleanup"

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Next Steps

Start the labs for this lesson, [Model Management Lab]($./Labs/05-Lab)

## Additional Topics & Resources

**Q:** Where can I find out more information on MLflow Model Registry?  
**A:** Check out <a href="https://mlflow.org/docs/latest/registry.html" target="_blank">the MLflow documentation</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>